In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card} Phase space factor cuts
TR-026
^^^
Visualization of the branch cuts of different phase space factor parametrizations. See also **[TR-003](https://compwa.github.io/report.003)**.
:::
::::

# Rotating square root cuts

In [ ]:
%pip install -q numpy==1.24.4

In [ ]:
from __future__ import annotations

from typing import Callable

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def plot_complex_plane(
    func: Callable[[np.ndarray], np.ndarray],
    *,
    x_range: tuple[float, float] = (-2, +5),
    y_range: tuple[float, float] = (-1, +1),
    vmax: float = 1.0,
    resolution: int = 500,
    function_name: str = R"\rho(s)",
    **func_kwargs,
) -> None:
    X, Y = np.meshgrid(
        np.linspace(*x_range, num=resolution),
        np.linspace(*y_range, num=resolution),
    )
    Z = X + Y * 1j
    T = func(Z, **func_kwargs)

    fig, axs = plt.subplots(
        figsize=(12, 5),
        gridspec_kw=dict(width_ratios=[1, 1, 0.05]),
        ncols=3,
        sharey=True,
    )
    ax1, ax2, cax = axs
    for ax in axs[:2]:
        ax.set_xlabel(R"$\mathrm{Re}\,s$")
    ax1.set_ylabel(R"$\mathrm{Im}\,s$")
    ax1.set_title(Rf"$\mathrm{{Re}}\,{function_name}$")
    ax2.set_title(Rf"$\mathrm{{Im}}\,{function_name}$")
    style = dict(cmap=plt.cm.coolwarm, vmin=-vmax, vmax=+vmax)
    mesh = ax1.pcolormesh(X, Y, T.real, **style)
    mesh = ax2.pcolormesh(X, Y, T.imag, **style)
    plt.colorbar(mesh, cax=cax)
    for ax in axs[:2]:
        ax.set_ylim(X.min(), X.max())
        ax.set_ylim(Y.min(), Y.max())
    fig.tight_layout()
    plt.show()

## Rotates square root

$$
\begin{eqnarray}
\sqrt[{[\phi]}]{z} &=& e^{\frac{1}{2}\phi i} \sqrt{z e^{-\phi i}}
\end{eqnarray}
$$

In [ ]:
def sqrt(z, phi: float = 0):
    return np.exp(1j * phi / 2) * np.sqrt(z * np.exp(-1j * phi))

In [ ]:
plot_complex_plane(
    sqrt,
    phi=np.pi / 6,
    x_range=(-1, +1),
    y_range=(-1, +1),
    function_name=R"\sqrt[{[\phi]}]{s}",
)

## PDG phase space factor

Phase space factor from the PDG as defined in Eq. (50.11), [PDG2023, §Resonances](https://pdg.lbl.gov/2023/reviews/rpp2023-rev-resonances.pdf#page=8):

$$
\begin{eqnarray}
\rho(s) &=& \frac{2\left|q(s)\right|}{\sqrt{s}} \\
q(s) &=& \frac{\sqrt{\lambda(s, m_1^2, m_2^2)}}{2\sqrt{s}} \\
\lambda(x, y, z) &=& x^2 + y^2 + z^2 - 2xy - 2yz - 2zx\end{eqnarray}
$$

In [ ]:
def phsp_factor_pdg(s, m1, m2, phi: float = 0):
    return 2 * breakup_momentum(s, m1, m2) / sqrt(s, phi)


def breakup_momentum(s, m1, m2):
    return np.sqrt(kallen(s, m1**2, m2**2)) / (2 * np.sqrt(s))


def kallen(x, y, z):
    return x**2 + y**2 + z**2 - 2 * x * y - 2 * y * z - 2 * z * x

In [ ]:
masses = dict(m1=0.1, m2=0.7)
plot_complex_plane(phsp_factor_pdg, **masses, phi=0)

## Wilson phase space factor

$$
\begin{eqnarray}
\rho(s) &=&
\sqrt{1-\frac{(m_1+m_2)^2}{s}}
\sqrt{1-\frac{(m_1-m_2)^2}{s}}
\end{eqnarray}
$$

In [ ]:
def phsp_wilson(s, m1, m2, phi: float = 0):
    return sqrt(1 - (m1 + m2) ** 2 / s, phi) * sqrt(1 - (m1 - m2) ** 2 / s, phi)

In [ ]:
plot_complex_plane(phsp_wilson, **masses, phi=0)

## Merged square root

$$
\begin{eqnarray}
\rho(s) &=&
\sqrt{
  \left(1-\frac{(m_1+m_2)^2}{s}\right)
  \left(1-\frac{(m_1-m_2)^2}{s}\right)
}
\end{eqnarray}
$$

In [ ]:
def phsp_merged_sqrt(s, m1, m2, phi: float = 0):
    return sqrt((1 - (m1 + m2) ** 2 / s) * (1 - (m1 - m2) ** 2 / s), phi)

In [ ]:
plot_complex_plane(phsp_merged_sqrt, **masses, phi=0)